In [5]:
import pandas as pd
df = pd.read_csv(r'C:\Users\bilobrkv\Downloads\C\Users\bilobrkv\Desktop\TINA -CC-DA\Portfolio\Paygab by industry.csv')

columns_to_drop = [
    'STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME',
    'Time frequency', 'Unit of measure',
    'nace_r2',
    'Geopolitical entity (reporting)', 'Time', 'Observation value',
    'Observation status (Flag) V2 structure', 'Confidentiality status (flag)'
]

df_cleaned = df.drop(columns=columns_to_drop)

df = df.rename(columns={
    'Statistical classification of economic activities in the European Community (NACE Rev. 2)': 'industry'
})
industry_gap = df.groupby('industry')['OBS_VALUE'].mean().sort_values(ascending=False)

industry_gap.head(10)


industry
Financial and insurance activities                                      28.075957
Wholesale and retail trade; repair of motor vehicles and motorcycles    20.387185
Information and communication                                           19.160510
Manufacturing                                                           19.135532
Professional, scientific and technical activities                       18.666958
Human health and social work activities                                 18.530189
Arts, entertainment and recreation                                      17.417568
Business economy                                                        16.977059
Other service activities                                                16.729688
Real estate activities                                                  14.149776
Name: OBS_VALUE, dtype: float64

In [13]:
import pdfplumber
import pandas as pd
import re

pdf_path = r'C:\Users\bilobrkv\Downloads\C\Users\bilobrkv\Desktop\TINA -CC-DA\Portfolio\sic2007.pdf'

sic_data = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        lines = text.split('\n')
        for line in lines:
            # Example pattern: "85310 General secondary education"
            parts = line.strip().split(' ', 1)
            if len(parts) == 2 and parts[0].isdigit():
                sic_data.append(parts)

# Convert to DataFrame
sic_df = pd.DataFrame(sic_data, columns=['sic_code', 'industry_name'])

sic_df.head()

,sic_code,industry_name
0,175,"Fifth Avenue, New York, NY 10010."
1,10,9 8 7 6 5 4 3 2 1
2,18,17 16 15 14 13 12 11 10 09
3,2007,2003 2007 2003
4,01160,01110 Abaca and other vegetable textile fibre ...


In [12]:
import pdfplumber
import pandas as pd
import re

pdf_path = r'C:\Users\bilobrkv\Downloads\C\Users\bilobrkv\Desktop\TINA -CC-DA\Portfolio\sic2007.pdf'

df1 = pd.read_csv(r'C:\Users\bilobrkv\Downloads\C\Users\bilobrkv\Desktop\TINA -CC-DA\Portfolio\UKpg.csv')

sic_data = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        lines = text.split('\n')
        for line in lines:
            # Example pattern: "85310 General secondary education"
            parts = line.strip().split(' ', 1)
            if len(parts) == 2 and parts[0].isdigit():
                sic_data.append(parts)

sic_data_df = pd.DataFrame(sic_data, columns=['sic_code', 'industry_name'])

df1['sic_primary'] = df1['sic_codes'].astype(str)

df1_merged = df1.merge(sic_data_df, left_on='sic_primary', right_on='sic_code', how='left')

# Preview merged data
df1_merged[['employer_name', 'sic_primary', 'industry_name']].head(20)
                                       

,employer_name,sic_primary,industry_name
0,"Bryanston School, Incorporated",85310,80210 Church schools at secondary level
1,"Bryanston School, Incorporated",85310,80210 Comprehensive schools
2,"Bryanston School, Incorporated",85310,80210 Grammar schools
3,"Bryanston School, Incorporated",85310,80210 Hospital schools at secondary level
4,"Bryanston School, Incorporated",85310,80210 Public schools
5,"Bryanston School, Incorporated",85310,80210 School examination board
6,"Bryanston School, Incorporated",85310,80210 Secondary level education
7,"Bryanston School, Incorporated",85310,80210 Secondary modern schools
8,"Bryanston School, Incorporated",85310,80210 Secondary schools
9,"Bryanston School, Incorporated",85310,80210 Sixth form colleges (manufacture)
